<a href="https://colab.research.google.com/github/Kridr/HW_PB_HSE/blob/main/Project_PB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install biopython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from Bio import SeqIO
from Bio import Entrez
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.SeqFeature import SeqFeature, FeatureLocation

In [ ]:
# Create a sequence
with open("fragment22.fasta") as handle:
    for record in SeqIO.parse(handle, "fasta"): # Там всего одна последовательность
        sequence_object = Seq(record.seq)

In [ ]:
import pandas as pd

In [ ]:
names = [
    'query acc.ver', 
    'subject acc.ver', 
    '% identity', 
    'alignment length', 
    'mismatches', 
    'gap opens', 
    'q. start', 
    'q. end', 
    's. start', 
    's. end', 
    'evalue', 
    'bit score', 
    '% positives'
]

uniprot_df = pd.read_csv('uniprot_blastp.csv', header=None, names=names)
pdb_df = pd.read_csv('pdb_blastp.csv', header=None, names=names)
refseq_select_prot_df = pd.read_csv('refseq_select_prot_blastp.csv', header=None, names=names)
pataa_df = pd.read_csv('pataa_blastp.csv', header=None, names=names)
refseq_protein_df = pd.read_csv('refseq_protein_blastp.csv', header=None, names=names)
tsa_nr_df = pd.read_csv('tsa_nr_blastp.csv', header=None, names=names)
env_nr_df = pd.read_csv('env_nr_blastp.csv', header=None, names=names)

uniprot_df.head()

,query acc.ver,subject acc.ver,% identity,alignment length,mismatches,gap opens,q. start,q. end,s. start,s. end,evalue,bit score,% positives
0,lcl|ORF49:31045:31941,Q966L9.1,44.144,111,50,3,170,276,69,171,3.290000e-05,48.9,48.65
1,lcl|ORF84:812:1537,P75853.2,38.889,198,113,4,47,241,27,219,4.740000e-42,148.0,60.10
2,lcl|ORF84:812:1537,P40400.1,29.208,202,133,5,36,231,20,217,3.150000e-17,82.4,50.50
3,lcl|ORF99:41885:42196,P42063.1,44.118,34,19,0,3,36,29,62,2.000000e-02,36.2,61.76
4,lcl|ORF100:42377:42619,P24139.1,42.373,59,34,0,2,60,108,166,2.890000e-05,43.1,62.71


In [ ]:
orfs = {}
with open('__all.fa') as f:
    seq = ''
    name = ''
    for line in f.readlines():
        if line and line[0] == '>':
            if seq and name:
                orfs[name] = seq
            seq = ''
            name = line.split()[0][1:]
        else:
            seq += line.strip()

In [244]:
# Create a record
record = SeqRecord(sequence_object,
                   id='fragement22', # random accession number
                   name='fragement22',
                   description='fragement22')
record.annotations = {'molecule_type': 'DNA'}

In [245]:
number = 0

In [246]:
s = set()
def add_features_from_df(df, identity_th=99, evalue_th = 1e-3):
    global record, number
    for i in df.index:
        if df.at[i, '% identity'] > identity_th and df.at[i, 'evalue'] < evalue_th:
            if df.at[i, 'query acc.ver'] in s:
                continue
            try:
                e = Entrez.efetch(db="biosample", id=uniprot_df['subject acc.ver'][120], retmode='text')
                e = e.read().split('\n')
                feature = SeqFeature(
                    FeatureLocation(start=int(df.at[i, 'q. start']), end=int(df.at[i, 'q. end'])),                
                    qualifiers={
                        'gene': e[0],
                        'product': df.at[i, 'subject acc.ver'],
                        'translation': orfs[df.at[i, 'query acc.ver']][df.at[i, 'q. start']:df.at[i, 'q. end']],
                        'note': f'{e[1]} # Identity: {df.at[i, "subject acc.ver"]} # Positives: {df.at[i, "% positives"]} # Evalue: {df.at[i, "evalue"]}',
                    }, 
                    type='misc_feature'
                )
                record.features.append(feature)
                number += 1
                s.add(df.at[i, 'query acc.ver'])
            except Exception as r: 
                print(r)

In [247]:
add_features_from_df(uniprot_df)
add_features_from_df(pdb_df)
add_features_from_df(refseq_select_prot_df)
add_features_from_df(pataa_df)
add_features_from_df(refseq_protein_df)
add_features_from_df(tsa_nr_df)
add_features_from_df(env_nr_df)

/usr/local/lib/python3.8/dist-packages/Bio/Entrez/__init__.py:686: UserWarning: 
            Email address is not specified.

            To make use of NCBI's E-utilities, NCBI requires you to specify your
            email address with each request.  As an example, if your email address
            is A.N.Other@example.com, you can specify it as follows:
               from Bio import Entrez
               Entrez.email = 'A.N.Other@example.com'
            In case of excessive usage of the E-utilities, NCBI will attempt to contact
            a user at the email address provided before blocking access to the
            E-utilities.
  warnings.warn(


In [248]:
# Save as GenBank file
with open('my.gb', 'w') as output_file:
    SeqIO.write(record, output_file, 'genbank')

In [249]:
number

4

In [250]:
s

{'lcl|ORF169:38333:29508',
 'lcl|ORF207:31459:31280',
 'lcl|ORF275:39729:38983',
 'lcl|ORF49:31045:31941'}